In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 13.2 MB/s 
     |████████████████████████████████| 9.3 MB 39.5 MB/s 
     |████████████████████████████████| 1.6 MB 51.8 MB/s 
     |████████████████████████████████| 108 kB 54.9 MB/s 


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [3]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [4]:
df.drop(['packer_type'],axis=1,inplace=True)

In [5]:
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [7]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [8]:
df.drop(['index'],axis=1,inplace=True)

In [9]:
df_5percent=df[0:2084] 


In [10]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,224,3,1,1,1,...,1,0,3,0,6.549930,0.519063,86313312,7.999864,1,0
1,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,6.271413,4.702766,1611776,6.571574,1,0
2,144,3,4,65535,184,224,4,1,0,1,...,1,1,3,0,5.948742,2.704501,194128,5.973493,1,0
3,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.450977,2.210593,112968,5.621887,1,0
4,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,6.015565,0.000000,1630208,5.885181,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,80,2,4,65535,184,216,6,1,1,1,...,1,4,2,0,0.000000,0.000000,218112,4.629545,1,1
2080,144,3,4,65535,184,280,4,1,0,1,...,1,1,3,0,6.042463,4.615791,178488,6.211368,1,0
2081,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,6.376993,0.000000,44448,6.423764,0,0
2082,144,3,4,65535,184,232,4,1,0,1,...,1,1,3,0,6.400777,1.863038,22528,5.880498,1,0


In [11]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (990, 69)
class 1: (1094, 69)


In [12]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [13]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     224.0               3.0   
1      144.0   3.0        4.0     65535.0  184.0     232.0               5.0   
2      144.0   3.0        4.0     65535.0  184.0     224.0               4.0   
3      144.0   3.0        4.0     65535.0  184.0     224.0               5.0   
4      144.0   3.0        4.0     65535.0  184.0     128.0               3.0   
...      ...   ...        ...         ...    ...       ...               ...   
2079    80.0   2.0        4.0     65535.0  184.0     216.0               6.0   
2080   144.0   3.0        4.0     65535.0  184.0     280.0               4.0   
2081   144.0   3.0        4.0     65535.0  184.0     128.0               3.0   
2082   144.0   3.0        4.0     65535.0  184.0     232.0               4.0   
2083   144.0   3.0        4.0     65535.0  184.0     224.0               5.0   

      C

In [14]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,2084.000000,2084.000000,2084.000000,2084.000000,2084.000000,2084.000000,2084.000000,2084.000000,2084.000000,2084.0,...,2084.000000,2084.000000,2084.000000,2084.000000,2084.000000,2084.000000,2.084000e+03,2084.000000,2084.000000,2084.000000
mean,164.977447,12.601727,10.131958,65049.738964,222.615163,222.667946,4.620921,0.992802,0.370441,1.0,...,0.999040,1.346449,3.274472,0.163148,4.951313,2.529415,8.276390e+05,6.376049,0.544626,0.524952
std,894.981345,439.148928,281.419590,5544.617875,1482.265027,50.060962,1.828526,0.084554,0.483039,0.0,...,0.030971,1.503375,1.163330,0.369589,2.520062,2.698194,5.831096e+06,1.112951,0.498124,0.499497
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.942896,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,3.912009,0.000000,6.540800e+04,5.713347,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.156149,1.574204,1.220640e+05,6.401573,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.505702,4.718681,2.969360e+05,7.309792,1.000000,1.000000
max,37008.000000,20050.000000,12851.000000,65535.000000,65534.000000,648.000000,29.000000,1.000000,1.000000,1.0,...,1.000000,24.000000,7.000000,1.000000,7.997171,7.999607,1.223571e+08,7.999997,1.000000,1.000000


In [15]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [16]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [17]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [18]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
0.0      55.0
1.0      45.0
dtype: float64

In [19]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
0.0      55.0
1.0      45.0
dtype: float64

In [20]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [21]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [22]:
circuit(weights_init, data[0])

tensor(0.0293889, requires_grad=True)

In [23]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias


In [24]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


In [25]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [26]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [27]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(0.01364634, requires_grad=False), tensor(-0.29807268, requires_grad=False)], Y = -1
X = [tensor(0.2741956, requires_grad=False), tensor(-0.14110237, requires_grad=False)], Y = -1
X = [tensor(0.00974093, requires_grad=False), tensor(-0.30317356, requires_grad=False)], Y = -1
X = [tensor(0.0886199, requires_grad=False), tensor(-0.38502744, requires_grad=False)], Y = -1
X = [tensor(-0.00163968, requires_grad=False), tensor(-0.31826926, requires_grad=False)], Y = -1


In [28]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [29]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.1395313 | Accuracy: 0.5312500 
Iter:     2 | Cost: 1.0890613 | Accuracy: 0.5125000 
Iter:     3 | Cost: 1.1854550 | Accuracy: 0.4125000 
Iter:     4 | Cost: 1.3962256 | Accuracy: 0.4125000 
Iter:     5 | Cost: 1.2517367 | Accuracy: 0.4125000 
Iter:     6 | Cost: 1.1574991 | Accuracy: 0.4125000 
New best
Iter:     7 | Cost: 1.0658654 | Accuracy: 0.6125000 
Iter:     8 | Cost: 1.0380376 | Accuracy: 0.5375000 
Iter:     9 | Cost: 1.0238238 | Accuracy: 0.5437500 
Iter:    10 | Cost: 1.0144057 | Accuracy: 0.5500000 
Iter:    11 | Cost: 1.0101292 | Accuracy: 0.5500000 
Iter:    12 | Cost: 1.0099685 | Accuracy: 0.5437500 
Iter:    13 | Cost: 1.0094639 | Accuracy: 0.5437500 
Iter:    14 | Cost: 0.9889273 | Accuracy: 0.5500000 
Iter:    15 | Cost: 0.9728137 | Accuracy: 0.5562500 
Iter:    16 | Cost: 0.9562175 | Accuracy: 0.5625000 
New best
Iter:    17 | Cost: 0.9598692 | Accuracy: 0.6812500 
Iter:    18 | Cost: 1.0035290 | Accuracy: 0.5062500 
Iter:    19 | Cost:

In [30]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [31]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.9265861625050945, Accuracy: 60.0%


In [32]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,1.0
1,1.0,-1.0
2,1.0,1.0
3,-1.0,-1.0
4,-1.0,-1.0
5,1.0,-1.0
6,1.0,-1.0
7,1.0,-1.0
8,1.0,1.0
9,1.0,1.0


In [33]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.27      1.00      0.43         6
         1.0       1.00      0.53      0.69        34

    accuracy                           0.60        40
   macro avg       0.64      0.76      0.56        40
weighted avg       0.89      0.60      0.65        40

1.0
0.5294117647058824
0.6923076923076924
0.7647058823529411
